In [12]:
import json
import openml
from scipy.stats import ttest_rel
import lightgbm as lgb
from src import DataBinner
import numpy as np
num_datasets = 18
num_models = 3
num_binning_methods = 3
num_seeds = 20

benchmark_suite = openml.study.get_suite(336) #337 for classification

for idx in range(18):
    if idx == 3:
        continue
    
    task_id = benchmark_suite.tasks[idx]
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    name = dataset.name
    obs = dataset.qualities['NumberOfInstances']
    features = dataset.qualities['NumberOfFeatures']
    print(f"===== DATASET {name} with {obs} observations and {features} features =====")    
    
    with open(f"../regression_results/regression_results_{idx}.json", "r") as f:
        results = json.load(f)
    
    minibatch_dict = results['minibatch_kmeans']
    linspace_dict = results['linspace']
    quantile_dict = results['none']
    
    for model in ['LGBM', 'CAT', 'XGB']:
        print(f"--- MODEL {model} ---")
        kmeans_results = minibatch_dict[model][1:]
        linspace_results = linspace_dict[model][1:]
        quantile_results = quantile_dict[model][1:]
        
        kmeans_mean = np.mean(kmeans_results)
        linspace_mean = np.mean(linspace_results)
        quantile_mean = np.mean(quantile_results)
        
        print(f"KMeans Relative Improvement: {100 * (quantile_mean - kmeans_mean) / quantile_mean:.2f}%")
        print(f"Linspace Relative Improvement: {100 * (quantile_mean - linspace_mean) / quantile_mean:.2f}%")
        
        t_stat, p_val = ttest_rel(kmeans_results, quantile_results)
        print(f"P-value for KMeans vs. Quantile: {p_val:.4f}. SIGNIFICANT? {p_val < 0.05}")
        
        t_stat, p_val = ttest_rel(linspace_results, quantile_results)
        print(f"P-value for Linspace vs. Quantile: {p_val:.4f}")
        
    
    

===== DATASET cpu_act with 8192.0 observations and 22.0 features =====
--- MODEL LGBM ---
KMeans Relative Improvement: 8.77%
Linspace Relative Improvement: 6.85%
P-value for KMeans vs. Quantile: 0.0000. SIGNIFICANT? True
P-value for Linspace vs. Quantile: 0.0128
--- MODEL CAT ---
KMeans Relative Improvement: -1.88%
Linspace Relative Improvement: -3.32%
P-value for KMeans vs. Quantile: 0.4538. SIGNIFICANT? False
P-value for Linspace vs. Quantile: 0.0962
--- MODEL XGB ---
KMeans Relative Improvement: -11.19%
Linspace Relative Improvement: -9.62%
P-value for KMeans vs. Quantile: 0.1065. SIGNIFICANT? False
P-value for Linspace vs. Quantile: 0.1055
===== DATASET pol with 15000.0 observations and 27.0 features =====
--- MODEL LGBM ---
KMeans Relative Improvement: -1.75%
Linspace Relative Improvement: -0.20%
P-value for KMeans vs. Quantile: 0.1101. SIGNIFICANT? False
P-value for Linspace vs. Quantile: 0.8314
--- MODEL CAT ---
KMeans Relative Improvement: 2.63%
Linspace Relative Improvement: -